In [79]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from math import sqrt

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [ ]:
  

class Head(nn.Module): 
    def __init__(self) -> None: 
        super().__init__() 
        self.query = nn.Linear(n_embd, n_embd // n_head)
        self.key = nn.Linear(n_embd, n_embd // n_head)
        self.value = nn.Linear(n_embd, n_embd // n_head)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) 

    def forward(self, matrix): 
        B, T, C = matrix.shape
        query = self.query(matrix) 
        key = self.key(matrix) 
        wei = query @ key.transpose(-2, -1) / sqrt(C) 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) 
        v = self.value(matrix) 
        return wei @ v
    
class MultiHeadAttention(nn.Module): 
    def __init__(self) -> None: 
        super().__init__()

        self.heads = nn.ModuleList([Head() for _ in range(n_head)])
        self.linear = nn.Linear(n_embd, n_embd)
    def forward(self, matrix): 
        temp = torch.cat([head(matrix) for head in self.heads], dim=-1)
        return self.linear(temp) 
    


class Block(nn.Module): 
    def __init__(self) -> None: 
        super().__init__() 

        self.multihead1 = MultiHeadAttention()
        self.norm1 = nn.LayerNorm(n_embd)
        
        self.multihead2 = MultiHeadAttention()
        self.norm2 = nn.LayerNorm(n_embd)

        self.ffw = nn.Linear(n_embd) 
        self.norm3 = nn.LayerNorm(n_embd)
        
    def forward(self, matrix): 

        x = self.multihead1(x)

        x = self.norm(matrix) 


class LanguageModel(nn.Module): 
    def __init__(self) -> None: 
        super().__init__() 

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) 
        self.position_embedding_table = nn.Embedding(vocab_size, n_embd) 
        self.head = MultiHeadAttention()


    def forward(self, xb, yb): 
        token = self.token_embedding_table(torch.tensor((xb)))
        position = self.position_embedding_table(xb)
        x = token + position 
        self.head(x)



model = LanguageModel() 
optimizer = torch.optim.AdamW(model.parameters(), learning_rate) 

for i in range(10): 
    xb, yb = get_batch('train')
    loss = model(xb, yb) 



torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])
torch.Size([16, 32, 16])
torch.Size([16, 32, 64])


/var/folders/zj/g8mnf8rd6rlc2z_s7v8msvsr0000gn/T/ipykernel_75133/3620252461.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  token = self.token_embedding_table(torch.tensor((xb)))
